In [13]:
import os
from tensorflow.keras.applications.xception import Xception, decode_predictions, preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.engine import Input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Reshape, Lambda, LSTM
from keras.layers import K
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
import tensorflow
import tensorflow as tf
import keras


import time
import numpy as np
from keras.utils import np_utils

np.random.seed(1337)


def load_val_dataset():
    validation_data_dir = '/home/divya/Documents/Minor_Project_Paper/CNN-RNN_implement/Potato/Valid'

    classes = []
    for subdir in sorted(os.listdir(validation_data_dir)):
        if os.path.isdir(os.path.join(validation_data_dir, subdir)):
            classes.append(subdir)

    class_indices = dict(zip(classes, range(len(classes))))

    X_val = []

    # Extracting validation dat
    i = 0
    y_val = []
    for subdir in classes:
        subpath = os.path.join(validation_data_dir, subdir)
        for fname in sorted(os.listdir(subpath)):
            y_val.append(class_indices[subdir])

            # Load image as numpy array and append it to X_val
            img = load_img(os.path.join(subpath, fname), target_size=(img_width, img_height))
            x = img_to_array(img)
            X_val.append(x)

            i += 1

    Y_val = np_utils.to_categorical(y_val)
    X_val = np.asarray(X_val, dtype='float32')
    return classes, X_val, Y_val


def rgb_to_grayscale(input):
    """Average out each pixel across its 3 RGB layers resulting in a grayscale image"""
    return K.mean(input, axis=3)


def rgb_to_grayscale_output_shape(input_shape):
    return input_shape[:-1]


nb_val_samples = 5000

img_width = 299
img_height = 299

print("Building model...")
input_tensor = tf.keras.Input(shape=(img_width, img_height, 3))

# Creating CNN
cnn_model = Xception(weights='imagenet', include_top=False, input_tensor=input_tensor)

x = cnn_model.output
cnn_bottleneck = tf.keras.layers.GlobalAveragePooling2D()(x)

# Creating RNN
x = Lambda(rgb_to_grayscale, rgb_to_grayscale_output_shape)(input_tensor)
x = Reshape((23, 3887))(x)  # 23 timesteps, input dim of each timestep 3887
x = LSTM(2048, return_sequences=True)(x)
rnn_output = LSTM(2048)(x)

# Merging both cnn bottleneck and rnn's output wise element wise multiplication
x = tf.keras.layers.Concatenate(axis=-1)([cnn_bottleneck, rnn_output])
predictions = Dense(3, activation='softmax')(x)

model = tf.keras.Model(inputs=input_tensor, outputs=predictions)

model.load_weights("/home/divya/Documents/Minor_Project_Paper/CNN-RNN_implement/cnn-rnn-classifier-master/finetuned_cnn_rnn_weights_2.hdf5")

print("Model built")

model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy',
              metrics=['accuracy', 'top_k_categorical_accuracy'])

classes, X_val, Y_val = load_val_dataset()

n_labels = len(classes)

n_imgs_by_label = np.zeros(n_labels, dtype=np.dtype(int))
n_top1_accurate_by_label = np.zeros(n_labels, dtype=np.dtype(int))
n_top5_accurate_by_label = np.zeros(n_labels, dtype=np.dtype(int))

# Loop over each validation image and calculate Top-1 and Top-5 Correct Classification Rate
for i, img in enumerate(X_val):
    print(i)
    ground_truth = Y_val[i].argmax()
    n_imgs_by_label[ground_truth] += 1

    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    preds = model.predict(img)

    top_5_indices = (-preds).argsort()[:, :5]
    top_5_indices = top_5_indices[0]
    if ground_truth == top_5_indices[0]:
        n_top1_accurate_by_label[ground_truth] += 1
    if ground_truth in top_5_indices:
        n_top5_accurate_by_label[ground_truth] += 1

# Create a text file that contains the top 1 and top 5 ACCR of each label
results_path = '/home/divya/Documents/Minor_Project_Paper/CNN-RNN_implement/cnn-rnn-classifier-master/cnn_rnn_results.txt'
with open(results_path, 'w+') as f:
    f.write('Label\tTop-1 Accuracy\tTop-5 Accuracy\n')
    for i, label in enumerate(classes):
        label_top1_accuracy = round(100.0 * n_top1_accurate_by_label[i] / n_imgs_by_label[i], 2)
        label_top5_accuracy = round(100.0 * n_top5_accurate_by_label[i] / n_imgs_by_label[i], 2)
        line = '{}\t{}\t{}\n'.format(label, label_top1_accuracy, label_top5_accuracy)
        f.write(line)


Building model...
Model built
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
